In [1]:
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
import json

In [2]:
class firefighterProblem:
    def __init__(self,fire_file, firefighter_file=None, node_file=None):
        #the section can be modified
        self.T_number = 150 #時間長度
        self.T = list([i for i in range(self.T_number + 1)]) #時間list
        self.P = {1:3,2:4} #各個消防單位時間處理的燃料量
        self.mode = "not random"
        
        self.model = gp.Model("FIREFIGHTER")
        self.M = 10
        self.epsilon=1e-4
        self.A_p = gp.tuplelist()
        self.A_f = gp.tuplelist()
        self.tau = gp.tupledict()
        self.lamb = gp.tupledict()
        self.K = set() #K=消防員集合
        self.Q = {}
        self.b = {}
        self.H = {}
        self.process = {}
        self.A_f_NEIGHBOR = {} #A_f_NEIGHBOR=與點i相鄰的點
        self.A_f_NEIGHBOR_T = {} #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點
        self.x = {}
        self.w = {}
        self.u = {}
        self.u_bar = {}
        self.v = {}
        self.v_bar = {}
        self.NODE_POS = {}
        
#        self.read_from_excel(node_file)
        self.read_from_excel(fire_file)
#         self.read_from_excel(firefighter_file)
        
    def read_from_excel(self, fileName):
            df = pd.read_excel(fileName, sheet_name = None)
            self.N_number = df["coordinates"].index[-1]+1
            self.N = set([i for i in range(1, self.N_number+1)])
            print(self.N)
            for i in df["fire_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                time = i['travel time']
                self.lamb[u,v] = time
                self.A_f.append((u,v))
            for i in self.N:
                self.A_p.append((i,i))
            for i in df["firefighter_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                firefighterIndex = i['k']
                if firefighterIndex not in self.K:
                    self.K.add(int(firefighterIndex))
                time = i['travel time']
                self.tau[u,v,firefighterIndex] = time
                if (u,v) not in self.A_p:
                    self.A_p.append((u,v)) 
            for i in range(self.N_number):
                self.NODE_POS[i+1] = (int(df["coordinates"].iloc[i]['x']), int(df["coordinates"].iloc[i]['y']))
            if self.mode == 'random':
                for i in range(self.N_number):
                    self.Q[i+1] = random.randint(2,5)
                    self.b[i+1] = random.randint(20,30)
                    self.H[i+1] = random.randint(2,5)
            else:
                for i in range(self.N_number):
                    self.Q[i+1] = int(df["coordinates"].iloc[i]['quantity'])
                    self.b[i+1] = int(df["coordinates"].iloc[i]['value'])
                    self.H[i+1] = int(df["coordinates"].iloc[i]['burning time'])
            print(len(self.H))
            self.N_D = df['ff_source']['N_D'].tolist()
            self.P = {1: int(df['ff_source']['P'].iloc[0])}
            self.N_F = set(df['fire_source']['N_F'].tolist())
            for i in self.N_D:
                if math.isnan(i):
                    self.N_D.remove(i)
            self.N_D = set(self.N_D)
        
        #         if "fire_route" in fileName:
#             fire_df = pd.read_excel(fileName)
#             df_num = len(fire_df.index)
#             for i in range(df_num):
#                 u = int(fire_df.iloc[i]['i'])
#                 v = int(fire_df.iloc[i]['j'])
#                 time = fire_df.iloc[i]['travel time']
#                 self.lamb[u,v] = time
#                 self.A_f.append((u,v))
#         elif "firefighter_route" in fileName:
#             firefighter_df = pd.read_excel(fileName)
#             df_num = len(firefighter_df.index)
#             for i in self.N:
#                 self.A_p.append((i,i))
#             for i in range(df_num):
#                 u = int(firefighter_df.iloc[i]['i'])
#                 v = int(firefighter_df.iloc[i]['j'])
#                 firefighterIndex = firefighter_df.iloc[i]['k']
#                 if firefighterIndex not in self.K:
#                     self.K.add(int(firefighterIndex))
#                 time = firefighter_df.iloc[i]['travel time']
#                 self.tau[u,v,firefighterIndex] = time
#                 #避免在多消防員時重複紀錄arc set
#                 if (u,v) not in self.A_p:
#                     self.A_p.append((u,v))  
#         elif "nodeInformation" in fileName:
#             nodeInfo1 = pd.read_excel(fileName, 'coordinates')
#             df_num = len(nodeInfo1.index)
#             self.N_number = df_num
#             self.N = set([i for i in range(1, self.N_number+1)])
            
#             for i in range(self.N_number):
#                 self.NODE_POS[i+1] = (int(nodeInfo1.iloc[i]['x']), int(nodeInfo1.iloc[i]['y']))
#             if self.mode == 'random':
#                 for i in range(self.N_number):
#                     self.Q[i+1] = random.randint(2,5)
#                     self.b[i+1] = random.randint(20,30)
#                     self.H[i+1] = random.randint(2,5)
#             else:
#                 for i in range(self.N_number):
#                     self.Q[i+1] = int(nodeInfo1.iloc[i]['quantity'])
#                     self.b[i+1] = int(nodeInfo1.iloc[i]['value'])
#                     self.H[i+1] = int(nodeInfo1.iloc[i]['burning time'])
            
#             nodeInfo2 = pd.read_excel(fileName, 'source')
#             self.N_D = set(nodeInfo2['N_D'].tolist())
#             self.N_F = set(nodeInfo2['N_F'].tolist())
    def initialize(self):
        for k in self.K:
            self.process[k]={}
            for i in self.N:
                if i in self.N_D:
                    self.process[k][i] = 0
                else:
                    self.process[k][i] = math.ceil(self.Q[i] * self.H[i] / self.P[k])
        for l in self.N - self.N_D:                          #定義A_f_NEIGHBOR
            connect = self.A_f.select('*',l)
            self.A_f_NEIGHBOR[l]=[]
            for temp in connect:
                self.A_f_NEIGHBOR[l].append(temp[0])
        for j in self.N - self.N_D - self.N_F:                         #定義A_f_NEIGHBOR_T
            for t in self.T:
                self.A_f_NEIGHBOR_T[j, t]=[]
                for i in self.A_f_NEIGHBOR[j]:
                    if t - self.H[i] - self.lamb[i, j]>=0:
                        self.A_f_NEIGHBOR_T[j, t].append(i)
        #定義x[i,j,k,t]
        for k in self.K:
            for t in self.T:
                for i in range(len(self.A_p)):
                    self.x[self.A_p[i][0], self.A_p[i][1], k, t] = self.model.addVar(vtype='B', name="x[%d,%d,%d,%d]" % (self.A_p[i][0], self.A_p[i][1], k, t))
        #定義w[i,k,t]
        for k in self.K:
            for t in self.T:
                for i in self.N:
                    self.w[i,k,t] = self.model.addVar(vtype='B',name="w[%d,%d,%d]" % (i, k, t))

        #定義u[i,t]
        self.u = self.model.addVars(self.N, self.T, vtype="B", name="u")

        #定義u_bar[i,k,t]
        self.u_bar = self.model.addVars(self.N, self.K, self.T, vtype="B", name="u_bar")

        #定義v[i,t]
        self.v = self.model.addVars(self.N, self.T, vtype="B", name="v")
        
        #定義v_bar[i,t]
        self.v_bar = self.model.addVars(self.N, self.T, vtype="B", name="v_bar")        
        
        self.model.update()
        
        #原點flow blance
        for k in self.K:                        
            self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i,j in self.A_p) <= 1)
    
        #限定從depot出發
        for O in self.N_D:
            connect = self.A_p.select(O,'*')
            for k in self.K:
                self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i, j in connect) == 1)
                
        #depot不會被保護
        #self.model.addConstrs((self.u_bar[i,k,t]==0 for i in self.N_D for k in self.K for t in range(self.T_number+1)))

        #flow balance
        for k in self.K:
            for t in range(1,self.T_number):
                for j in self.N: 
                    in_connect = self.A_p.select('*',j)
                    out_connect = self.A_p.select(j,'*')
                    temp = 0 #in-degree
                    temp += self.w[j, k, t - 1] # t-1在j idle
                    
                    if j in self.N_D: #j in depot set會有u_bar，只是都為0
                        temp += self.u_bar[j, k, t]
                    else: #j not in depot set, 若現在的t > process time，則有u_bar且為非0
                        if self.process[k][j] <= t:
                            temp += self.u_bar[j, k, t - self.process[k][j]]
                    for m, n in in_connect:
                        if m != n and self.tau[m, n, k] <= t: #若現在的t>travel time，則會有x
                            temp += self.x[m, n, k, t - self.tau[m,n,k]]
                    self.model.addConstr(temp == gp.quicksum(self.x[n, w, k, t] for n, w in out_connect), name="flow") #in-degree = out-degree
        
        #若在t from i to i(i not include depot), 一定會是在t時刻開始保護或idle
        self.model.addConstrs((self.u_bar[i, k, t] + self.w[i, k, t] == self.x[i, i, k, t] for i in self.N - self.N_D for k in self.K for t in range(self.T_number)))
        
        #若在t from s to s(s is in deopt set), 一定會在t時刻idle
        self.model.addConstrs((self.w[s, k, t] == self.x[s, s, k, t]) for s in self.N_D for k in self.K for t in range(self.T_number))
        
        #每個node在T內只會開始燒、開始保護、或未影響
        self.model.addConstrs(self.u[i, t] + self.u_bar.sum(i, '*', t) <= 1 for i in self.N for t in self.T)
        
        #開始燒與已經燒之間的關係
        self.model.addConstrs(self.v[i, t] + self.u[i, t] == self.v[i, t+1] for i in self.N for t in self.T[0:-1])
        
        #開始保護與已經保護之間的關係
        self.model.addConstrs(self.v_bar[i, t] + self.u_bar.sum(i, '*', t) == self.v_bar[i, t+1] for i in self.N for t in self.T[0:-1]) #constrain 9
        
        #deopt在T內不會開始保護
        self.model.addConstrs(self.u_bar.sum(s, '*', '*') == 0 for s in self.N_D)
        
        #depot在T內不會開始燒
        self.model.addConstrs(self.u.sum(s, '*') == 0 for s in self.N_D)
        
        #火焰的延燒
        for j in self.N - self.N_D - self.N_F:
            for t in range(self.T_number):
                if len(self.A_f_NEIGHBOR_T[j, t]) == 0:
                    self.model.addConstr(self.u[j, t] == 0, name='test')
                else:
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) / self.M <= self.u[j, t] + self.v[j, t] + self.v_bar[j, t + 1])
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) >= self.u[j, t])
        
        #給定起火點
        for i in self.N_F:                           
            self.model.addConstr(self.u[i, 0] == 1)
        
        #給定所有節點已經燒的起始狀態
        for i in self.N:                           
            self.model.addConstr(self.v[i, 0] == 0)
    
        #給定所有節點已經保護的起始狀態
        for i in self.N:                            
            self.model.addConstr(self.v_bar[i, 0] == 0)

        #消防員不能去已經被燃燒的節點
        for k in self.K:                             
            for t in self.T:
                for l in range(len(self.A_p)):
                    if self.A_p[l][1] not in self.N_D:
                        if self.A_p[l][0] ==  self.A_p[l][1]:
                            if t + 2 <= self.T_number:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + 1]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                            else:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        elif t + self.tau[self.A_p[l][0], self.A_p[l][1], k] + 1 <= self.T_number:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + self.tau[self.A_p[l][0], self.A_p[l][1], k]]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        else:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
        
        #設定目標式
        self.model.setObjective(gp.quicksum(gp.quicksum(self.u[i, t] for t in self.T) * self.b[i] for i in self.N - self.N_D) + 
                           gp.quicksum(self.epsilon * self.x[i, j, k, t] for (i, j, k, t) in self.x if i != j) +
                           gp.quicksum(self.epsilon * t * self.u_bar[i, k, t] for i in self.N - self.N_D for k in self.K for t in self.T), GRB.MINIMIZE)
        return self.model
        
    def showTextSol(self):
        print("x:")
        for k in self.K:
            print()
            print("消防員%d的路徑" % k)
            temp = [elem for elem in self.x if elem[2] == k]
            for (i, j, k, t) in temp:
                if self.x[i, j, k, t].X > self.epsilon:
                    if i != j:
                        print("在時刻 %d 從node%d 移動到 node%d" % (t, i, j), " ,travel time:", self.tau[i, j, k])
                    else:            
                        if self.u_bar[i,k,t].X == 1:
                            print("在時刻 %d 對node%d進行保護" % (t,i), " ,processing time:", self.process[k][i])
                        else:
                            print("在時刻 %d 在node%d idle" % (t,i))

#         print("w:")
#         for (i, k, t) in self.w:
#             if self.w[i, k, t].X > self.epsilon:
#                 print("w[%d,%d,%d]" % (i, k, t) , self.w[i, k, t].X)

#         print("u:")
#         for (i, t) in self.u:
#             if self.u[i,t].X > self.epsilon:
#                 print("u[%d,%d]" % (i,t), self.u[i,t].X)

#         print("u_bar:")
#         for (i, k, t) in self.u_bar:
#             if self.u_bar[i, k, t].X > self.epsilon:
#                 print("u_bar[%d,%d,%d]" % (i, k, t), self.u_bar[i, k, t].X)

#         print("v:")
#         for (i, t) in self.v:
#             if self.v[i, t].X > self.epsilon:
#                 print("v[%d,%d]" % (i,t), self.v[i, t].X)

#         print("v_bar:")
#         for (i, t) in self.v_bar:
#             if self.v_bar[i, t].X > self.epsilon:
#                 print("v_bar[%d,%d]" % (i,t), self.v_bar[i, t].X)
    def writeJson(self, file):
        data = {}
        data['NODE_POS'] = self.NODE_POS
        data['N'] = list(self.N)
        data['N_D'] = list(self.N_D)
        data['N_F'] = list(self.N_F)
        data['K'] = list(self.K)
        data['A_p'] = list([str(i) for i in self.A_p])
        data['A_f'] = list([str(i) for i in self.A_f])
        data['tau'] = dict((str(i), self.tau[i]) for i in self.tau)
        data['lamb'] = dict((str(i), self.lamb[i]) for i in self.lamb)
        data['T'] = self.T
        data['q'] = self.Q
        data['b'] = self.b
        data['p'] = self.P
        data['h'] = self.H
        
        temp = {}
        for (i, j, k, t) in self.x:
            temp[str((i, j, k, t))] = self.x[i, j, k, t].X
        data['x'] = temp

        temp = {}
        for (i, k, t) in self.w:
            temp[str((i, k, t))] = self.w[i, k, t].X
        data['w'] = temp

        temp = {}
        for (i, t) in self.u:
            temp[str((i, t))] = self.u[i, t].X
        data['u'] = temp

        temp = {}
        for (i, k, t) in self.u_bar:
            temp[str((i, k, t))] = self.u_bar[i, k, t].X
        data['u_bar'] = temp

        temp = {}
        for (i, t) in self.v:
            temp[str((i, t))] = self.v[i, t].X
        data['v']  = temp

        temp = {}
        for (i, t) in self.v_bar:
            temp[str((i, t))] = self.v_bar[i, t].X
        data['v_bar'] = temp
        
        json_data = json.dumps(data)
        with open(file[:-5]+"_data.json", "w") as file:
            file.write(json.dumps(data))

In [3]:
import os
files = os.listdir("./")

file_list = [f for f in files if os.path.isfile(os.path.join("./", f))and f[-5:] == ".xlsx"]

for file in file_list:
    print(file)

    ff = firefighterProblem(fire_file=file)
    model = ff.initialize()

    #run model and write lp file
    model.optimize()
    #model.write('test.lp')
    print("optimal value : ", model.ObjVal)

    ff.showTextSol()
    ff.writeJson(file)

FFP_n20_no4.xlsx
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-26
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 31507 rows, 27482 columns and 109402 nonzeros
Model fingerprint: 0x9d88c339
Variable types: 0 continuous, 27482 integer (27482 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 22850 rows and 16565 columns
Presolve time: 0.58s
Presolved: 8657 rows, 10917 columns, 34687 nonzeros
Variable types: 0 continuous, 10917 integer (10917 binary)
Found heuristic solution: objective 170.0025000

Root relaxation: objective 7.619131e+01, 4843 iterations, 0.42 seconds (0.42 work units)

    Nodes    |    Current Node    |     Objective Bo

在時刻 114 在node16 idle
在時刻 115 在node16 idle
在時刻 116 在node16 idle
在時刻 117 在node16 idle
在時刻 118 在node16 idle
在時刻 119 在node16 idle
在時刻 120 在node16 idle
在時刻 121 在node16 idle
在時刻 122 在node16 idle
在時刻 123 在node16 idle
在時刻 124 在node16 idle
在時刻 125 在node16 idle
在時刻 126 在node16 idle
在時刻 127 在node16 idle
在時刻 128 在node16 idle
在時刻 129 在node16 idle
在時刻 130 在node16 idle
在時刻 131 在node16 idle
在時刻 132 在node16 idle
在時刻 133 在node16 idle
在時刻 134 在node16 idle
在時刻 135 在node16 idle
在時刻 136 在node16 idle
在時刻 137 在node16 idle
在時刻 138 在node16 idle
在時刻 139 在node16 idle
在時刻 140 在node16 idle
在時刻 141 在node16 idle
在時刻 142 在node16 idle
在時刻 143 在node16 idle
在時刻 144 在node16 idle
在時刻 145 在node16 idle
在時刻 146 在node16 idle
在時刻 147 在node16 idle
在時刻 148 在node16 idle
在時刻 149 在node16 idle
G30_xin_model_1.xlsx
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
30
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors,

  1860   976  191.60235   31 1337  200.03060  182.57751  8.73%   238  206s
  1863   981  182.57751   16 1113  200.03060  182.57751  8.73%   256  210s
  1883   982     cutoff   19       200.03060  182.57751  8.73%   262  215s
  1901   989  183.95447   21  762  200.03060  182.87787  8.58%   266  220s
  1927   989  184.00738   24  845  200.03060  183.95478  8.04%   268  225s
  1961   988  195.16001   26  612  200.03060  183.95478  8.04%   273  230s
  2003   988  185.97407   30  839  200.03060  183.95478  8.04%   279  236s
  2045   989     cutoff   32       200.03060  183.95478  8.04%   285  241s
  2089  1003  186.88800   35  741  200.03060  183.95478  8.04%   287  246s
  2137  1011  187.17541   37  586  200.03060  183.95478  8.04%   286  250s
  2178  1014     cutoff   39       200.03060  183.95478  8.04%   286  255s
  2245  1032  190.63056   44  611  200.03060  183.95478  8.04%   284  260s
  2298  1033  192.87154   49  434  200.03060  183.95478  8.04%   284  266s
  2354  1051  194.76987  

     0     0   58.05967    0  796  185.03660   58.05967  68.6%     -   16s
     0     0   58.22093    0  820  185.03660   58.22093  68.5%     -   16s
     0     0   58.22516    0  826  185.03660   58.22516  68.5%     -   16s
     0     0   58.22532    0  833  185.03660   58.22532  68.5%     -   16s
     0     0   58.25429    0  827  185.03660   58.25429  68.5%     -   17s
     0     0   58.25436    0  831  185.03660   58.25436  68.5%     -   17s
     0     0   58.25648    0  783  185.03660   58.25648  68.5%     -   17s
     0     0   58.25655    0  787  185.03660   58.25655  68.5%     -   17s
H    0     0                     175.0431000   58.25655  66.7%     -   17s
     0     0   58.26423    0  786  175.04310   58.26423  66.7%     -   17s
     0     0   58.26423    0  778  175.04310   58.26423  66.7%     -   18s
H    0     2                     175.0430000   58.97320  66.3%     -   20s
     0     2   58.97320    0  778  175.04300   58.97320  66.3%     -   20s
    11    16   58.97320  

H    0     0                     160.0287000   88.26619  44.8%     -   10s
     0     0   88.26627    0  654  160.02870   88.26627  44.8%     -   10s
H    0     0                     155.0262000   88.39396  43.0%     -   10s
     0     0   88.39396    0  579  155.02620   88.39396  43.0%     -   10s
H    0     0                     155.0260000   88.45135  42.9%     -   11s
     0     0   88.45135    0  536  155.02600   88.45135  42.9%     -   11s
     0     0   88.53661    0  535  155.02600   88.53661  42.9%     -   11s
H    0     0                     155.0259000   88.53661  42.9%     -   11s
     0     0   88.53663    0  535  155.02590   88.53663  42.9%     -   11s
H    0     0                     155.0250000   88.80507  42.7%     -   11s
     0     0   88.80507    0  521  155.02500   88.80507  42.7%     -   11s
     0     0   88.89461    0  704  155.02500   88.89461  42.7%     -   12s
     0     0   88.94730    0  628  155.02500   88.94730  42.6%     -   12s
     0     0   88.94794  

  3366  1235  131.02065   31  611  150.03050  120.74970  19.5%   368  397s
  3430  1252  133.73603   35  653  150.03050  120.74970  19.5%   364  403s
  3494  1306  135.04205   48  482  150.03050  122.14581  18.6%   366  425s
  3592  1291     cutoff   30       150.03050  123.35141  17.8%   362  431s
  3672  1276 infeasible   95       150.03050  123.72878  17.5%   366  439s
  3739  1257  140.04365   29  276  150.03050  124.22530  17.2%   371  446s
  3800  1233 infeasible  104       150.03050  124.82585  16.8%   378  454s
  3857  1240 infeasible  100       150.03050  125.04232  16.7%   387  464s
  3972  1199  139.77195   33  550  150.03050  125.04232  16.7%   389  472s
  4049  1222 infeasible  102       150.03050  127.78137  14.8%   390  480s
  4162  1247     cutoff   33       150.03050  127.78137  14.8%   392  490s
  4316  1254  132.05925   41  595  150.03050  127.78137  14.8%   389  500s
  4415  1262  140.04707   44  435  150.03050  127.78137  14.8%   393  514s
  4494  1259  140.05018  

   271   153  163.25263   56  410  190.02680  157.47951  17.1%   380   56s
H  316   180                     190.0267000  157.47951  17.1%   351   59s
   320   214  163.59954   73  384  190.02670  157.47951  17.1%   357   60s
   413   249  164.04519   80  445  190.02670  157.47951  17.1%   339   65s
   551   344  165.10265  105  391  190.02670  157.56446  17.1%   289   71s
   618   393     cutoff   10       190.02670  159.70852  16.0%   295   75s
H  670   393                     190.0266000  159.70905  16.0%   285   75s
   733   432  161.75407   15  523  190.02660  159.70905  16.0%   282   80s
   787   427     cutoff   11       190.02660  160.75822  15.4%   319   86s
   832   463  187.89895   32  207  190.02660  162.13742  14.7%   354   91s
   923   506  167.87601   34  383  190.02660  162.34731  14.6%   352   96s
  1003   511  179.71167   61  271  190.02660  162.72279  14.4%   365  101s
  1027   519  172.05330    8  203  190.02660  162.81899  14.3%   376  112s
  1069   532  180.87122  

Model fingerprint: 0x1f9bf4df
Variable types: 0 continuous, 48924 integer (48924 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 42279 rows and 28161 columns
Presolve time: 1.21s
Presolved: 13512 rows, 20763 columns, 60413 nonzeros
Variable types: 0 continuous, 20763 integer (20763 binary)
Found heuristic solution: objective 260.0032000

Root relaxation: objective 9.629892e+01, 14540 iterations, 3.13 seconds (3.25 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   96.29892    0  322  260.00320   96.29892  63.0%     -    4s
     0     0  105.81603    0  501  260.00320  105.81603  59.3%     -    6s
H    0     0                     195.0188000  105.81603  45.7%     -    6s
H    0     0                     170.0253000  105.81603 

   963   325 infeasible   99       145.02100  131.27795  9.48%   482  197s
  1005   354     cutoff  108       145.02100  131.33911  9.43%   482  201s
  1071   382     cutoff  110       145.02100  131.47299  9.34%   470  205s
  1145   358     cutoff   42       145.02100  131.81984  9.10%   457  210s
  1173   366  135.12112   37  946  145.02100  132.23009  8.82%   465  215s
  1238   391  139.91743   48  952  145.02100  132.29639  8.77%   458  220s
  1407   253     cutoff   59       145.02100  137.48793  5.19%   437  229s
  1491    49 infeasible   67       145.02100  139.67439  3.69%   431  232s

Cutting planes:
  Gomory: 5
  Lift-and-project: 18
  Cover: 6
  Clique: 35
  MIR: 3
  Zero half: 126
  RLT: 37

Explored 1744 nodes (722426 simplex iterations) in 233.29 seconds (219.17 work units)
Thread count was 8 (of 8 available processors)

Solution count 8: 145.021 145.021 145.021 ... 260.003

Optimal solution found (tolerance 1.00e-04)
Best objective 1.450210000000e+02, best bound 1.450210

     0     0   92.93381    0  935  160.03540   92.93381  41.9%     -   18s
     0     0   92.93382    0  940  160.03540   92.93382  41.9%     -   18s
     0     0   92.99525    0  955  160.03540   92.99525  41.9%     -   19s
     0     0   93.01140    0  929  160.03540   93.01140  41.9%     -   19s
     0     0   93.01145    0  945  160.03540   93.01145  41.9%     -   19s
     0     0   93.14926    0  926  160.03540   93.14926  41.8%     -   19s
     0     0   93.15686    0  928  160.03540   93.15686  41.8%     -   20s
     0     0   93.16096    0  925  160.03540   93.16096  41.8%     -   20s
     0     0   93.16097    0  926  160.03540   93.16097  41.8%     -   20s
     0     0   93.17506    0  841  160.03540   93.17506  41.8%     -   20s
     0     0   93.17508    0  844  160.03540   93.17508  41.8%     -   20s
     0     0   93.17515    0  856  160.03540   93.17515  41.8%     -   20s
     0     0   93.17515    0  843  160.03540   93.17515  41.8%     -   21s
H    0     0             

In [3]:
ff = firefighterProblem(fire_file="FFP_n20_no3.xlsx")
model = ff.initialize()

#run model and write lp file
model.optimize()
#model.write('test.lp')
print("optimal value : ", model.ObjVal)

ff.showTextSol()
ff.writeJson(file)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-26
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20


KeyError: 2